In [1]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium
from itertools import product 
from sklearn.preprocessing import LabelEncoder

In [9]:
# 데이터 불러오기
train_5 = pd.read_pickle('./train_5.pkl')
train_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8860578 entries, 0 to 8860577
Data columns (total 18 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   month_id                  int8   
 1   shop_id                   int8   
 2   item_id                   int16  
 3   cnt_month                 float32
 4   price_mean                float32
 5   city_id                   int8   
 6   cat_id                    int8   
 7   main_cat                  int8   
 8   sub_cat                   int8   
 9   year                      int16  
 10  month                     int8   
 11  first_sell                int8   
 12  sales_month               int8   
 13  item_id_mean_cnt          float32
 14  sub_cat_mean_cnt          float32
 15  price_mean_mean_cnt       float32
 16  item_id/city_id_mean_cnt  float32
 17  shop_id/cat_id_mean_cnt   float32
dtypes: float32(7), int16(2), int8(9)
memory usage: 414.1 MB


In [11]:
train_5.isnull().sum()

month_id                         0
shop_id                          0
item_id                          0
cnt_month                   214200
price_mean                  214200
city_id                          0
cat_id                           0
main_cat                         0
sub_cat                          0
year                             0
month                            0
first_sell                       0
sales_month                      0
item_id_mean_cnt            214200
sub_cat_mean_cnt            214200
price_mean_mean_cnt         214200
item_id/city_id_mean_cnt    214200
shop_id/cat_id_mean_cnt     214200
dtype: int64

In [47]:
# 각 매장/상품 별로 임의로 정한 개월 수 이전의 값으로 컬럼을 추가하는 함수 정의
# num_lags=2라면 1개월전의 값 컬럼과 2개월전의 값 컬럼이 생성됨
# rolling()과 같은 과정이지만 현 데이터에 rolling을 적용하기 복잡하기 때문에 함수로 정의해서 사용

def make_lags(data, feature, num_lags):
    temp = data[['month_id','shop_id','item_id',feature]].copy() # 원본은 건드리지 않도록 copy()

    # lag 피쳐 생성 및 기존 데이터와 병합
    for i in range(num_lags):
        column_name = feature+'_'+str(i+1) # 새로 생성될 컬럼명 지정 
        temp.columns = ['month_id','shop_id','item_id',column_name] # temp 컬럼명 새 이름으로 재지정
        temp['month_id'] = temp['month_id']+(i+1) # month_id에 해당 lag 숫자(몇 개월 전인지)를 더하기 > 해당 숫자만큼 월이 밀린 데이터가 됨 
        data = data.merge(temp, on=['month_id','shop_id','item_id'], how='left')
        data[column_name] = data[column_name].fillna(0) # 판매이력이 없는 경우(신상품) NaN이 존재할 수 있으므로 결측치는 0으로 대체
    del temp
    return data

In [54]:
# 월 판매량 lags 추가
train_6 = make_lags(train_5, 'cnt_month', 3)
train_6.head()

,month_id,shop_id,item_id,cnt_month,price_mean,city_id,cat_id,main_cat,sub_cat,year,...,first_sell,sales_month,item_id_mean_cnt,sub_cat_mean_cnt,price_mean_mean_cnt,item_id/city_id_mean_cnt,shop_id/cat_id_mean_cnt,cnt_month_1,cnt_month_2,cnt_month_3
0,0,59,22154,1.0,999.0,30,37,11,1,2013,...,0,0,0.457143,0.208571,2.783668,1.0,0.096774,0.0,0.0,0.0
1,0,59,2552,0.0,0.0,30,58,13,29,2013,...,0,0,0.000000,0.046831,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,30,58,13,29,2013,...,0,0,0.028571,0.046831,0.000000,0.0,0.000000,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,30,56,13,5,2013,...,0,0,0.028571,0.060099,0.000000,0.0,0.011494,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,30,59,13,40,2013,...,0,0,0.142857,0.096500,0.000000,0.0,0.013245,0.0,0.0,0.0


In [55]:
# 이전에 추가했던 기준별 평균 판매량에 대해 lags 추가
train_6 = make_lags(train_6, 'item_id_mean_cnt', 3)
train_6 = make_lags(train_6, 'sub_cat_mean_cnt', 3)
train_6 = make_lags(train_6, 'price_mean_mean_cnt', 3)
train_6 = make_lags(train_6, 'item_id/city_id_mean_cnt', 3)
train_6 = make_lags(train_6, 'shop_id/cat_id_mean_cnt', 3)
train_6.tail()

,month_id,shop_id,item_id,cnt_month,price_mean,city_id,cat_id,main_cat,sub_cat,year,...,sub_cat_mean_cnt_3,price_mean_mean_cnt_1,price_mean_mean_cnt_2,price_mean_mean_cnt_3,item_id/city_id_mean_cnt_1,item_id/city_id_mean_cnt_2,item_id/city_id_mean_cnt_3,shop_id/cat_id_mean_cnt_1,shop_id/cat_id_mean_cnt_2,shop_id/cat_id_mean_cnt_3
8860573,34,45,18454,NaN,NaN,20,55,13,4,2015,...,0.207317,1.545279,0.0,0.0,0.5,0.0,0.0,0.126866,0.106061,0.128242
8860574,34,45,16188,NaN,NaN,20,64,14,42,2015,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.094488,0.000000,0.000000
8860575,34,45,15757,NaN,NaN,20,55,13,4,2015,...,0.207317,0.000000,0.0,0.0,0.0,0.0,0.0,0.126866,0.106061,0.128242
8860576,34,45,19648,NaN,NaN,20,40,11,6,2015,...,0.231667,0.000000,0.0,0.0,0.0,0.0,0.0,0.083756,0.112455,0.083426
8860577,34,45,969,NaN,NaN,20,37,11,1,2015,...,0.151524,0.000000,0.0,0.0,0.5,0.0,0.0,0.128205,0.124088,0.093750


In [56]:
# month_id 0,1,2 데이터 제거 > 3개월 만큼 lags 생성했기때문에 해당값이 없음
train_6 = train_6.drop(train_6[train_6['month_id'] < 3].index)

In [58]:
# 데이터 저장
train_6.to_pickle('./train_6.pkl')

In [62]:
# 테스트 데이터에 존재하지 않을 데이터 컬럼 제거
train_6 = train_6.drop(['price_mean','item_id_mean_cnt','sub_cat_mean_cnt','price_mean_mean_cnt',
                        'item_id/city_id_mean_cnt','shop_id/cat_id_mean_cnt'],axis=1)

In [64]:
train_6 = train_6.drop(['first_sell'],axis=1)

In [63]:
train_6.isnull().sum()

month_id                           0
shop_id                            0
item_id                            0
cnt_month                     214200
city_id                            0
cat_id                             0
main_cat                           0
sub_cat                            0
year                               0
month                              0
first_sell                         0
sales_month                        0
cnt_month_1                        0
cnt_month_2                        0
cnt_month_3                        0
item_id_mean_cnt_1                 0
item_id_mean_cnt_2                 0
item_id_mean_cnt_3                 0
sub_cat_mean_cnt_1                 0
sub_cat_mean_cnt_2                 0
sub_cat_mean_cnt_3                 0
price_mean_mean_cnt_1              0
price_mean_mean_cnt_2              0
price_mean_mean_cnt_3              0
item_id/city_id_mean_cnt_1         0
item_id/city_id_mean_cnt_2         0
item_id/city_id_mean_cnt_3         0
s

In [68]:
# 0 ~ 20 사이로 값 제한
train_6[['cnt_month', 'cnt_month_1','cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1','item_id_mean_cnt_2', 'item_id_mean_cnt_3', 
        'sub_cat_mean_cnt_1','sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3','item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1','shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3']] = train_6[['cnt_month', 'cnt_month_1','cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1','item_id_mean_cnt_2', 'item_id_mean_cnt_3', 
        'sub_cat_mean_cnt_1','sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3','item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1','shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3']].clip(0, 20)

In [67]:
train_6.columns

Index(['month_id', 'shop_id', 'item_id', 'cnt_month', 'city_id', 'cat_id',
       'main_cat', 'sub_cat', 'year', 'month', 'sales_month', 'cnt_month_1',
       'cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1',
       'item_id_mean_cnt_2', 'item_id_mean_cnt_3', 'sub_cat_mean_cnt_1',
       'sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3',
       'item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1',
       'shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3'],
      dtype='object')

In [69]:
# 훈련 데이터 (피처)
X_train = train_6[train_6['month_id'] < 33]
X_train = X_train.drop(['cnt_month'], axis=1)
# 검증 데이터 (피처)
X_valid = train_6[train_6['month_id'] == 33]
X_valid = X_valid.drop(['cnt_month'], axis=1)
# 테스트 데이터 (피처)
X_test = train_6[train_6['month_id'] == 34]
X_test = X_test.drop(['cnt_month'], axis=1)

# 훈련 데이터 (타깃 값)
y_train = train_6[train_6['month_id'] < 33]['cnt_month']
# 검증 데이터 (타깃 값)
y_valid = train_6[train_6['month_id'] == 33]['cnt_month']

In [70]:
import lightgbm as lgb

# LightGBM 하이퍼 파라미터
params = {'metric': 'rmse', # 평가지표 = rmse
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 42}

cat_features = ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat']

# LightGBM 훈련 및 검증 데이터세트
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)
 
# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)

[LightGBM] [Info] Total Bins 4050
[LightGBM] [Info] Number of data points in the train set: 7596885, number of used features: 28
[LightGBM] [Info] Start training from score 0.306474
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.02071	valid_1's rmse: 0.914527
[200]	training's rmse: 0.918663	valid_1's rmse: 0.851745
[300]	training's rmse: 0.867585	valid_1's rmse: 0.829806
[400]	training's rmse: 0.839642	valid_1's rmse: 0.824203
[500]	training's rmse: 0.82206	valid_1's rmse: 0.823819
Early stopping, best iteration is:
[436]	training's rmse: 0.832361	valid_1's rmse: 0.823313


In [73]:
df_test = pd.read_csv('e:/Git_public_dodo_Riley/kaggle/Future_Sales/data/test.csv')
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [74]:
# 예
preds = lgb_model.predict(X_test).clip(0, 20)



submission = pd.DataFrame({
    "ID": df_test.index, 
    "item_cnt_month": preds
})
submission.to_csv('lgbm_submission.csv', index=False)

In [ ]:
# Score: 0.90263, 2352/13591~상위 17.3%